# CIFAR-10

## LOAD the dataset

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.datasets import cifar10

In [ ]:
data_train, data_test = cifar10.load_data()

(images, labels) = data_train

(images_train, labels_train) = images[:40000], labels[:40000]
(images_validation, labels_validation) = images[40000:], labels[40000:]

(images_test, labels_test) = data_test

labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
n_class = len(labels)

In [ ]:
print(images_train.shape)
print(labels_train.shape)
print(images_validation.shape)
print(labels_validation.shape)
print(images_test.shape)
print(labels_test.shape)

In [ ]:
print(labels_test[1])

In [ ]:
for i in range(10):
  cv2_imshow(images_train[i])

## Create a deep learning model

In [ ]:
# TODO: Implement a CNN model
'''
train_data: (n_train, 32, 32, 3)
train_labels: (n_train, 1)
n_class: 10

Use the AI chatbot to help you. Build a CNN model for the CIFAR-10 dataset.
You don't need an overly large model. You can import whatever you need.
'''
'''Your code here'''


In [ ]:
def train_model(model, train_data, train_labels=None, epochs=10, batch_size=32,
                criterion=None, optimizer=None, lr=1e-3):
    import tensorflow as tf
    import torch
    from tqdm import tqdm, trange

    tf_gpu_available = tf.config.list_physical_devices('GPU')
    torch_gpu_available = torch.cuda.is_available()

    if isinstance(model, tf.keras.Model):
        device = "GPU" if tf_gpu_available else "CPU"
        print(f"[INFO] Detected TensorFlow model. Using {device}.")

        if optimizer is None:
            optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size)

    elif isinstance(model, torch.nn.Module):
        device = torch.device("cuda" if torch_gpu_available else "cpu")
        print(f"[INFO] Detected PyTorch model. Using {device}.")
        model.to(device)
        model.train()

        if isinstance(train_data, np.ndarray):
            train_data = torch.tensor(train_data, dtype=torch.float32)
            if train_data.ndim == 4:
                train_data = train_data.permute(0, 3, 1, 2)

        if isinstance(train_labels, np.ndarray):
            train_labels = torch.tensor(train_labels, dtype=torch.long)
        if train_labels.ndim == 2 and train_labels.shape[1] == 1:
            train_labels = train_labels.squeeze(1)

        from torch.utils.data import TensorDataset, DataLoader
        dataset = TensorDataset(train_data, train_labels)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        if optimizer is None:
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        if criterion is None:
            criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(epochs):
            total_loss = 0
            for x_batch, y_batch in tqdm(loader, desc='Batch'):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)

                optimizer.zero_grad()
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

    else:
        raise TypeError("Check the model type!")


In [ ]:
train_model(model, images_train, labels_train, epochs=5, batch_size=64, lr=0.001)

## Train

## Test

In [ ]:
def predict_model(model, input_data):
    import tensorflow as tf
    import torch
    import numpy as np

    if isinstance(model, tf.keras.Model):
        return model.predict(input_data)

    elif isinstance(model, torch.nn.Module):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.eval()
        model.to(device)

        if isinstance(input_data, np.ndarray):
            input_tensor = torch.from_numpy(input_data).float()

            if input_tensor.ndim == 4 and input_tensor.shape[3] == 3:
                input_tensor = input_tensor.permute(0, 3, 1, 2)
            elif input_tensor.ndim == 3 and input_tensor.shape[2] == 3:
                input_tensor = input_tensor.permute(2, 0, 1).unsqueeze(0)

        else:
            input_tensor = input_data

        if input_tensor.ndim == 3:
            input_tensor = input_tensor.unsqueeze(0)

        input_tensor = input_tensor.to(device)

        with torch.no_grad():
            output = model(input_tensor)
            output = torch.softmax(output, dim=1)

        return output.cpu().numpy()

    else:
        raise TypeError("Check the model type!")

In [ ]:
test_ind = 0 # 0~9999

result = predict_model(model, images_test[test_ind].reshape(1, 32, 32, 3))

print("Truth: " + labels[int(labels_test[test_ind])] + " / Pred: "+ labels[np.argmax(result)])
img = cv2.resize(images_test[test_ind]*255, (80, 80))
cv2_imshow(img)

In [ ]:
def evaluate_model(model, test_data, test_labels, criterion=None):
    import tensorflow as tf
    import torch
    import numpy as np

    if isinstance(model, tf.keras.Model):
        print("[INFO] Evaluating TensorFlow model...")
        return model.evaluate(test_data, test_labels, verbose=0)

    elif isinstance(model, torch.nn.Module):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()

        if isinstance(test_data, np.ndarray):
            x_tensor = torch.from_numpy(test_data).float()

            if x_tensor.ndim == 4 and x_tensor.shape[3] == 3:
                x_tensor = x_tensor.permute(0, 3, 1, 2)
        else:
            x_tensor = test_data

        if isinstance(test_labels, np.ndarray):
            y_tensor = torch.from_numpy(test_labels).long()
        else:
            y_tensor = test_labels

        if y_tensor.ndim == 2 and y_tensor.shape[1] == 1:
            y_tensor = y_tensor.squeeze(1)

        x_tensor, y_tensor = x_tensor.to(device), y_tensor.to(device)

        if criterion is None:
            criterion = torch.nn.CrossEntropyLoss()

        with torch.no_grad():
            outputs = model(x_tensor)
            loss = criterion(outputs, y_tensor)
            preds = torch.argmax(outputs, dim=1)
            accuracy = (preds == y_tensor).float().mean().item()

        return loss.item(), accuracy

    else:
        raise TypeError("Check the model type!")

In [ ]:
test_loss, test_acc = evaluate_model(model, images_test, labels_test)
print('test accuracy: {:.4f}'.format(test_acc))